<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for accessing Forecastingdata feature extraction tools using python

The feature calculation and other utilities are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook shows an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
runs directly on python. Part2 is an example of how to use the tools directly from python.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [3]:
#@title
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Clone the repository, for reproducibility. No need to clone everytime if you have already have the folder!.

In [5]:
!git clone https://github.com/pmontman/TSForecasting

fatal: destination path 'TSForecasting' already exists and is not an empty directory.


In [25]:
%cd TSForecasting/
!git pull
%cd ..

/content/TSForecasting
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
From https://github.com/pmontman/TSForecasting
   37a5613..ee0510a  master     -> origin/master
Updating 37a5613..ee0510a
Fast-forward
 experiments/fixed_horizon.R | 22 +++++++++++++---------
 models/global_models.R      |  5 ++++-
 2 files changed, 17 insertions(+), 10 deletions(-)
/content


We now prepare the R environment for processing: installing and loading the required packages

In [6]:
%%R
devtools::install_github("hendersontrent/catch22")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
install.packages("smooth")
library("Rcatch22")

R[write to console]: Skipping install of 'Rcatch22' from a github remote, the SHA1 (a93d1a9b) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/tsibble_1.0.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1517998 bytes (1.4 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

Install catboost R package for the tree-based forecasts

In [7]:
%%R
install.packages('glmnet')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/glmnet_4.1-2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2184440 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

In [8]:
%%R

source("TSForecasting/experiments/feature_experiments.R")

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



The fixed horizon we need to try line by line to skip errors

In [13]:
%%R
version
#source("TSForecasting/experiments/fixed_horizon.R")
ll <- parse(file = "TSForecasting/experiments/fixed_horizon.R")

for (i in seq_along(ll)) {
    tryCatch(eval(ll[[i]]), 
             error = function(e) message("Oops!  ", as.character(e)))
}

R[write to console]: Oops!  Error: package or namespace load failed for ‘catboost’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/usr/local/lib/R/site-library/catboost/libs/libcatboostr.so':
  /usr/local/lib/R/site-library/catboost/libs/libcatboostr.so: undefined symbol: R_DimSymbol




[1] "Started loading sample"
[1] "started Forecasting"
[1] 1


R[write to console]: Oops!  Error in eval(expr, p): object 'xreg' not found




[1] "Started loading sample"
[1] "started Forecasting"


R[write to console]: Oops!  Error in start_forecasting(train_series_list, lag, forecast_horizon, method): could not find function "start_forecasting"




...

R[write to console]: Oops!  Error in file(file, "r"): cannot open the connection




[1] "Started loading aus_elecdemand"


R[write to console]: Oops!  Error in file(file, "r"): cannot open the connection




[1] "Started loading rideshare"


R[write to console]: Oops!  Error in file(file, "r"): cannot open the connection




In [26]:
%%R
BASE_DIR <- "TSForecasting"

source(file.path(BASE_DIR, "utils", "data_loader.R", fsep = "/"))
source(file.path(BASE_DIR, "utils", "error_calculator.R", fsep = "/"))
source(file.path(BASE_DIR, "utils", "global_model_helper.R", fsep = "/"))
source(file.path(BASE_DIR, "models", "local_univariate_models.R", fsep = "/"))
source(file.path(BASE_DIR, "models", "global_models.R", fsep = "/"))


In [27]:
%%R
# The name of the column containing time series values after loading data from the .tsf file into a tsibble
VALUE_COL_NAME <- "series_value"

# Defining the required forecasting models based on frequency. 
# Current supporting forecasting methods are ets, theta, simple exponential smoothing, tbats, auto.arima and dynamic harmonic regression arima
# You can also define new forecasting methods (probably in models/local_univariate_models.R) and link them here as required
MODELS_HIGH_FREQ <- c("ses", "theta", "ets", "arima") # for yearly, quarterly, monthly, and daily datasets
MODELS_LOW_FREQ <- c("ses", "theta", "tbats", "dhr_arima") # for 4_seconds, minutely, 10_minutes, 15_minutes, half_hourly, hourly and weekly datasets

# Seasonality values corresponding with the frequencies: 4_seconds, minutely, 10_minutes, 15_minutes, half_hourly, hourly, daily, weekly, monthly, quarterly and yearly
# Consider multiple seasonalities for frequencies less than daily
SEASONALITY_VALS <- list()
SEASONALITY_VALS[[1]] <- c(21600, 151200, 7889400)
SEASONALITY_VALS[[2]] <- c(1440, 10080, 525960)
SEASONALITY_VALS[[3]] <- c(144, 1008, 52596)
SEASONALITY_VALS[[4]] <- c(96, 672, 35064)
SEASONALITY_VALS[[5]] <- c(48, 336, 17532)
SEASONALITY_VALS[[6]] <- c(24, 168, 8766)
SEASONALITY_VALS[[7]] <- 7
SEASONALITY_VALS[[8]] <- 365.25/7
SEASONALITY_VALS[[9]] <- 12 
SEASONALITY_VALS[[10]] <- 4
SEASONALITY_VALS[[11]] <- 1  

SEASONALITY_MAP <- list()

for(f in seq_along(FREQUENCIES)){
  SEASONALITY_MAP[[FREQUENCIES[f]]] <- SEASONALITY_VALS[[f]]
}


# This function performs the fixed horizon evaluation with local models
#
# Parameters
# dataset_name - the name of the dataset
# method - a list of forecasting models that needs to be evaluated
# input_file_name - name of the .tsf file corresponding with the dataset
# key - the name of the attribute that should be used as the key when creating the tsibble
# index - the name of the time attribute that should be used as the index when creating the tsibble
# external_forecast_horizon - the required forecast horizon, if it is not available in the .tsf file
# integer_conversion - whether the forecasts should be rounded or not
do_fixed_horizon_local_forecasting <- function(dataset_name, methods, input_file_name, key = NULL, index = NULL, external_forecast_horizon = NULL, integer_conversion = FALSE){
  
  print(paste0("Started loading ", dataset_name))
  
  # Loading data from the .tsf file
  loaded_data <- convert_tsf_to_tsibble(file.path(BASE_DIR, "tsf_data", input_file_name, fsep = "/"), VALUE_COL_NAME, key, index)
  dataset <- loaded_data[[1]]
  frequency <- loaded_data[[2]]
  forecast_horizon <- loaded_data[[3]]
  
  
  if(!is.null(frequency))
    seasonality <- SEASONALITY_MAP[[frequency]]
  else
    seasonality <- 1
  
  
  # If the forecast horizon is not given within the .tsf file, then it should be provided as a function input
  if(is.null(forecast_horizon) & !("horizon" %in% colnames(dataset))){
    if(is.null(external_forecast_horizon))
      stop("Please provide the required forecast horizon")
    else
      forecast_horizon <- external_forecast_horizon
  }
  
  all_serie_names <- unique(dataset$series_name)
  
  train_series_list <- list()
  
  if(!is.null(forecast_horizon))
    actual_matrix <- matrix(NA, nrow = length(all_serie_names), ncol = forecast_horizon)
  else
    actual_matrix <- matrix(NA, nrow = length(all_serie_names), ncol = max(as.numeric(dataset[["horizon"]])))
  
  
  start_time <- Sys.time()
  
  print("started Forecasting")
  
  dir.create(file.path(BASE_DIR, "results", "fixed_horizon_forecasts", fsep = "/"), showWarnings = FALSE, recursive=TRUE)
  
  for(s in seq_along(all_serie_names)){
    print(s)
    
    series_data <- dataset[dataset$series_name == as.character(all_serie_names[s]), ]
    
    if("horizon" %in% colnames(dataset))
      forecast_horizon <- as.numeric(unique(series_data[["horizon"]]))
    
    if(nrow(series_data) < forecast_horizon)
      forecast_horizon <- 1
    
    train_series_data <- series_data[1:(nrow(series_data) - forecast_horizon),]
    test_series_data <- series_data[(nrow(series_data) - forecast_horizon + 1):nrow(series_data),][[VALUE_COL_NAME]]
    
    if(forecast_horizon < ncol(actual_matrix))
      test_series_data <- c(test_series_data, rep(NA, (ncol(actual_matrix) - forecast_horizon)))
    
    train_series_list[[s]] <- train_series_data[[VALUE_COL_NAME]]
    actual_matrix[s,] <- test_series_data
    
    if(!is.null(index))
      start_date <- start(as.ts(train_series_data[, c(index, VALUE_COL_NAME)], frequency = max(seasonality)))
    
    if(is.null(index))
      series <- forecast:::msts(train_series_data[[VALUE_COL_NAME]], seasonal.periods = seasonality)
    else
      series <- forecast:::msts(train_series_data[[VALUE_COL_NAME]], start = start_date, seasonal.periods = seasonality)
    
    # Forecasting
    for(method in methods){
      current_method_forecasts <- eval(parse(text = paste0("get_", method, "_forecasts(series, forecast_horizon)")))
      
      if(typeof(current_method_forecasts) == "list")
        current_method_forecasts <- current_method_forecasts[[1]]
      
      current_method_forecasts[is.na(current_method_forecasts)] <- 0
      
      if(integer_conversion)
        current_method_forecasts <- round(current_method_forecasts)
      
      write.table(t(c(all_serie_names[s], current_method_forecasts)), file.path(BASE_DIR, "results", "fixed_horizon_forecasts", paste0(dataset_name, "_", method, ".txt"), fsep = "/"), row.names = FALSE, col.names = FALSE, sep = ",", quote = FALSE, append = TRUE)
    }
  }
  
  end_time <- Sys.time()
  
  print("Finished Forecasting")
  
  # Execution time
  exec_time <- end_time - start_time
  print(exec_time)
  dir.create(file.path(BASE_DIR, "results", "fixed_horizon_execution_times", fsep = "/"), showWarnings = FALSE, recursive=TRUE)
  write(paste(exec_time, attr(exec_time, "units")), file = file.path(BASE_DIR, "results", "fixed_horizon_execution_times", paste0(dataset_name, ".txt"), fsep = "/"), append = FALSE)
  
  # Error calculations
  dir.create(file.path(BASE_DIR, "results", "fixed_horizon_errors", fsep = "/"), showWarnings = FALSE, recursive=TRUE)
  
  for(method in methods){
    forecast_matrix <- read.csv(file.path(BASE_DIR, "results", "fixed_horizon_forecasts", paste0(dataset_name, "_", method, ".txt"), fsep = "/"), header = F)
    forecast_matrix <- as.matrix(forecast_matrix[-1])
    calculate_errors(forecast_matrix, actual_matrix, train_series_list, seasonality, file.path(BASE_DIR, "results", "fixed_horizon_errors", paste0(dataset_name, "_", method), fsep = "/"))
  }
}



# This function performs the fixed horizon evaluation with global models
# For our experiments, we only consider pooled regression under this category
#
# Parameters
# dataset_name - the name of the dataset
# lag - the number of past lags that should be used when predicting the next future value of time series
# input_file_name - name of the .tsf file corresponding with the dataset
# method - name of the global forecasting method that you want to evaluate
# key - the name of the attribute that should be used as the key when creating the tsibble
# index - the name of the time attribute that should be used as the index when creating the tsibble
# external_forecast_horizon - the required forecast horizon, if it is not available in the .tsf file
# integer_conversion - whether the forecasts should be rounded or not
do_fixed_horizon_global_forecasting <- function(dataset_name, lag, input_file_name, method, key = NULL, index = NULL, external_forecast_horizon = NULL, integer_conversion = FALSE){
  
  print(paste0("Started loading ", dataset_name))
  
  # Loading data from the .tsf file
  loaded_data <- convert_tsf_to_tsibble(file.path(BASE_DIR, "tsf_data", input_file_name, fsep = "/"), VALUE_COL_NAME, key, index)
  dataset <- loaded_data[[1]]
  frequency <- loaded_data[[2]]
  forecast_horizon <- loaded_data[[3]]
  
  
  if(!is.null(frequency))
    seasonality <- SEASONALITY_MAP[[frequency]]
  else
    seasonality <- 1
  
  
  if("horizon" %in% colnames(dataset))
    forecast_horizon <- max(as.numeric(dataset[["horizon"]]))
  
  # If the forecast horizon is not given within the .tsf file, then it should be provided as a function input  
  if(is.null(forecast_horizon)){
    if(is.null(external_forecast_horizon))
      stop("Please provide the required forecast horizon")
    else
      forecast_horizon <- external_forecast_horizon
  }
  
  all_serie_names <- unique(dataset$series_name)
  
  train_series_list <- list()
  
  actual_matrix <- matrix(NA, nrow = length(all_serie_names), ncol = forecast_horizon)
  
  start_time <- Sys.time()
  
  print("started Forecasting")
  
  for(s in seq_along(all_serie_names)){
    series_data <- dataset[dataset$series_name == as.character(all_serie_names[s]), ]
    
    if("horizon" %in% colnames(dataset))
      series_forecast_horizon <- as.numeric(unique(series_data[["horizon"]]))
    else
      series_forecast_horizon <- forecast_horizon
    
    train_series_data <- series_data[1:(nrow(series_data) - series_forecast_horizon),]
    test_series_data <- series_data[(nrow(series_data) - series_forecast_horizon + 1):nrow(series_data),][[VALUE_COL_NAME]]
    
    if(series_forecast_horizon < ncol(actual_matrix))
      test_series_data <- c(test_series_data, rep(NA, (ncol(actual_matrix) - series_forecast_horizon)))
    
    train_series_list[[s]] <- train_series_data[[VALUE_COL_NAME]]
    actual_matrix[s,] <- test_series_data
  }
  
  # Forecasting
  forecast_matrix <- start_forecasting(train_series_list, lag, forecast_horizon, method)
  forecast_matrix[is.na(forecast_matrix)] <- 0
  
  if(integer_conversion)
    forecast_matrix <- round(forecast_matrix)
  
  dir.create(file.path(BASE_DIR, "results", "fixed_horizon_forecasts", fsep = "/"), showWarnings = FALSE, recursive=TRUE)
  
  for(s in seq_along(all_serie_names)){
    actual_series <- as.numeric(actual_matrix[s,])
    diff <- sum(is.na(actual_series))
    
    if(diff > 0)
      forecast_matrix[s, (ncol(forecast_matrix) - diff + 1) : ncol(forecast_matrix)] <- rep(NA, diff)
    
    write.table(t(c(all_serie_names[s], as.numeric(forecast_matrix[s,])[1:(length(actual_series) - diff)])), file.path(BASE_DIR, "results", "fixed_horizon_forecasts", paste0(dataset_name, "_", method, "_lag_", lag, ".txt"), fsep = "/"), row.names = FALSE, col.names = FALSE, sep = ",", quote = FALSE, append = TRUE)
  }
  
  end_time <- Sys.time()
  
  print("Finished Forecasting")
  
  # Execution time
  exec_time <- end_time - start_time
  print(exec_time)
  dir.create(file.path(BASE_DIR, "results", "fixed_horizon_execution_times", fsep = "/"), showWarnings = FALSE, recursive=TRUE)
  write(paste(exec_time, attr(exec_time, "units")), file = file.path(BASE_DIR, "results", "fixed_horizon_execution_times", paste0(dataset_name, "_", method, "_lag_", lag, ".txt"), fsep = "/"), append = FALSE)
  
  # Error calculations
  dir.create(file.path(BASE_DIR, "results", "fixed_horizon_errors", fsep = "/"), showWarnings = FALSE, recursive=TRUE)
  forecast_matrix <- as.matrix(forecast_matrix)
  calculate_errors(forecast_matrix, actual_matrix, train_series_list, seasonality, file.path(BASE_DIR, "results", "fixed_horizon_errors", paste0(dataset_name, "_", method, "_lag_", lag), fsep = "/"))
}


In [ ]:
%%R

do_fixed_horizon_local_forecasting("sample", MODELS_LOW_FREQ, "sample.tsf", "series_name", "start_timestamp", 8)

[1] "Started loading sample"
[1] "started Forecasting"
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 1

## Part 2) Calculating features directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [10]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [11]:
%%capture cap --no-stderr
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("sample", "sample.tsf", "series_name", "start_timestamp", "catch22")

The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [12]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/sample_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/sample_features.csv

head of the tsfeatures features
sample_features.csv
mean,var,max_kl_shift,time_kl_shift,max_level_shift,time_level_shift,max_var_shift,time_var_shift,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,ARCH.LM,crossing_points,entropy,flat_spots,alpha,beta,hurst,lumpiness,nonlinearity,x_pacf5,diff1x_pacf5,diff2x_pacf5,seas_pacf,stability,unitroot_kpss,unitroot_pp,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,peak,trough
105.652455128205,669.530932765715,1.2706668176004,52,0.755290510889679,2,0.959745807190754,2,0.710805797539225,1.51175743379058,-0.239716429411997,0.105249143633781,-0.539721534779089,0.372057773793558,-0.0890329975757608,0.257725362872141,22,0.795329846811295,14,0.628482857552452,0.000100053599224661,0.989518146632079,0.254142008742879,0.315470866286382,0.530751963124808,0.104820622256713,0.656220390192105,0.0363289043636947,0.160377933409522,0.459213533792908,-39.7197694054302,1,52,0.233768779821994,0.00021